In [23]:
# Step 1: Import Libraries
import torch # pytorch package import
import torch.nn as nn
import torch.optim as optim
import torchvision  # pytorch vision package import
from  torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("mps" if torch.mps.is_available() else "cpu")

In [17]:
# Paths to dataset directories
train_dir = "./data/FER2013Train"
val_dir = "./data/FER2013Valid"
test_dir = "./data/FER2013Test"

# Paths to labels
train_csv = "./data/FER2013Train/label.csv"
val_csv = "./data/FER2013Valid/label.csv"
test_csv = "./data/FER2013Test/label.csv"

In [18]:
# Step 2: Load and Preprocess the Dataset
# ----------------------------------------
# Fill in the blanks to load and preprocess the dataset
# Hint: Use torchvision.datasets for datasets like CIFAR-10 or MNIST

# Define transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(size=(32, 32)),  # Resize if necessary
    transforms.ToTensor() ,  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,), (0.5,),)  # Normalize the images
])

from FERPlusDataset import FERPlusDataset


train_dataset = FERPlusDataset(image_dir=train_dir, transform=transform, label_csv=train_csv, mode="majority")
val_dataset = FERPlusDataset(image_dir=val_dir, transform=transform, label_csv=val_csv, mode="majority")
test_dataset = FERPlusDataset(image_dir=test_dir, transform=transform, label_csv=test_csv, mode="majority")

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)  # Fix in_channels
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=64 * 8 * 8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Activation and pooling
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = CNN().to(device)

In [26]:


criterion = nn.CrossEntropyLoss()  # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # call optimizer

In [27]:
# Step 5: Train the Model
num_epochs = 10 # define number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:  # Print every 100 batches
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

Epoch [1/10], Step [100/296], Loss: 1.5665
Epoch [1/10], Step [200/296], Loss: 1.5072
Epoch [2/10], Step [100/296], Loss: 1.4280
Epoch [2/10], Step [200/296], Loss: 1.3840
Epoch [3/10], Step [100/296], Loss: 1.3677
Epoch [3/10], Step [200/296], Loss: 1.3364
Epoch [4/10], Step [100/296], Loss: 1.2899
Epoch [4/10], Step [200/296], Loss: 1.2794
Epoch [5/10], Step [100/296], Loss: 1.2440
Epoch [5/10], Step [200/296], Loss: 1.2363
Epoch [6/10], Step [100/296], Loss: 1.2058
Epoch [6/10], Step [200/296], Loss: 1.1956
Epoch [7/10], Step [100/296], Loss: 1.1856
Epoch [7/10], Step [200/296], Loss: 1.1514
Epoch [8/10], Step [100/296], Loss: 1.1404
Epoch [8/10], Step [200/296], Loss: 1.1463
Epoch [9/10], Step [100/296], Loss: 1.1244
Epoch [9/10], Step [200/296], Loss: 1.1189
Epoch [10/10], Step [100/296], Loss: 1.1270
Epoch [10/10], Step [200/296], Loss: 1.1088


In [28]:
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")
    return test_acc

test_acc = test(model, test_loader, device)


Test Accuracy: 59.70%
